In [27]:
import pandas as pd
import numpy as np
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# df = pd.read_csv("data/reviews.xlsx", delimiter=';', encoding='latin')
df = pd.read_excel("data/reviews.xlsx")
# texts = df["cerita"].tolist()
# labels = df["genre"].tolist()

data = df[['Reviews', 'Label']]
data.head()

,Reviews,Label
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1
4,Kain nya bagus halus \nTapi kok di bukak koto...,0


In [3]:
list(data.columns.values)

number_of_cerita = data.Reviews.count()
data_counts = df.Label.value_counts()

print(number_of_cerita)
print(data_counts)

831
Label
0    446
1    385
Name: count, dtype: int64


Case Folding

In [5]:
data['Case_Folding'] = df['Reviews'].str.lower()
data

,Reviews,Label,Case_Folding
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0,"jahitannya sih rapi,cuman ada benang yang ikut..."
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0,sesuai harga. agak tipis tapi masih oke kok. w...
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1,"wah gila sihhh sebagus itu, se worth it, se l..."
4,Kain nya bagus halus \nTapi kok di bukak koto...,0,kain nya bagus halus \ntapi kok di bukak koto...
...,...,...,...
826,Terima kasih barang sudah sampai sesuai ukuran...,1,terima kasih barang sudah sampai sesuai ukuran...
827,Mantapp realpicttt bangttt tapi pengemasan nya...,1,mantapp realpicttt bangttt tapi pengemasan nya...
828,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1,"suka bgt sama tasnya, ga kayak tas local. kere..."
829,kualitas produk sangat baik. produk original. ...,1,kualitas produk sangat baik. produk original. ...


Cleansing

In [6]:
import string

def cleansing(txt):
    txt = txt.encode('ascii', 'replace').decode('ascii')
    txt = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", txt).split())
    txt = re.sub(r"\d+", "", txt).replace('?', ' ').replace(',', ' ')
    txt = txt.translate(str.maketrans("-"," ",string.punctuation))
    txt = txt.strip()
    txt = re.sub('\s+',' ',txt)
    txt = re.sub(r"\b[a-zA-Z]\b", "", txt)
    return txt

data['Cleansing'] = data['Case_Folding'].apply(cleansing)
data
    

,Reviews,Label,Case_Folding,Cleansing
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,kemeja nya bagusss bgtttt aaaa mauuu nngisssss...
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0,"jahitannya sih rapi,cuman ada benang yang ikut...",jahitannya sih rapi cuman ada benang yang ikut...
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0,sesuai harga. agak tipis tapi masih oke kok. w...,sesuai harga agak tipis tapi masih oke kok war...
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1,"wah gila sihhh sebagus itu, se worth it, se l...",wah gila sihhh sebagus itu se worth it se lemb...
4,Kain nya bagus halus \nTapi kok di bukak koto...,0,kain nya bagus halus \ntapi kok di bukak koto...,kain nya bagus halus tapi kok di bukak kotor y...
...,...,...,...,...
826,Terima kasih barang sudah sampai sesuai ukuran...,1,terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukuran...
827,Mantapp realpicttt bangttt tapi pengemasan nya...,1,mantapp realpicttt bangttt tapi pengemasan nya...,mantapp realpicttt bangttt tapi pengemasan nya...
828,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1,"suka bgt sama tasnya, ga kayak tas local. kere...",suka bgt sama tasnya ga kayak tas local keren ...
829,kualitas produk sangat baik. produk original. ...,1,kualitas produk sangat baik. produk original. ...,kualitas produk sangat baik produk original ha...


Tokenizing

In [79]:
data['Tokenize'] = data['Cleansing'].str.split()
data

,Reviews,Label,Case_Folding,Cleansing,Tokenize
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,kemeja nya bagusss bgtttt aaaa mauuu nngisssss...,"[kemeja, nya, bagusss, bgtttt, aaaa, mauuu, nn..."
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0,"jahitannya sih rapi,cuman ada benang yang ikut...",jahitannya sih rapi cuman ada benang yang ikut...,"[jahitannya, sih, rapi, cuman, ada, benang, ya..."
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0,sesuai harga. agak tipis tapi masih oke kok. w...,sesuai harga agak tipis tapi masih oke kok war...,"[sesuai, harga, agak, tipis, tapi, masih, oke,..."
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1,"wah gila sihhh sebagus itu, se worth it, se l...",wah gila sihhh sebagus itu se worth it se lemb...,"[wah, gila, sihhh, sebagus, itu, se, worth, it..."
4,Kain nya bagus halus \nTapi kok di bukak koto...,0,kain nya bagus halus \ntapi kok di bukak koto...,kain nya bagus halus tapi kok di bukak kotor y...,"[kain, nya, bagus, halus, tapi, kok, di, bukak..."
...,...,...,...,...,...
826,Terima kasih barang sudah sampai sesuai ukuran...,1,terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukuran...,"[terima, kasih, barang, sudah, sampai, sesuai,..."
827,Mantapp realpicttt bangttt tapi pengemasan nya...,1,mantapp realpicttt bangttt tapi pengemasan nya...,mantapp realpicttt bangttt tapi pengemasan nya...,"[mantapp, realpicttt, bangttt, tapi, pengemasa..."
828,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1,"suka bgt sama tasnya, ga kayak tas local. kere...",suka bgt sama tasnya ga kayak tas local keren ...,"[suka, bgt, sama, tasnya, ga, kayak, tas, loca..."
829,kualitas produk sangat baik. produk original. ...,1,kualitas produk sangat baik. produk original. ...,kualitas produk sangat baik produk original ha...,"[kualitas, produk, sangat, baik, produk, origi..."


In [80]:
spill_words = {
    "": ["aaaa"],
    "kecil": ["kecillll"],
    "nangis": ["nngisssss", "huhuuuu"],
    "si": ["siiiii"],
    "bohong": ["bhong"],
    "bagus": ["baklgus", "bagusssssssssssssssssssssssss", "bgaaaaaaaaus", "bagusssssssss", "bagusssssss", "baaaaaguuuuussss"],
    "banget": ["bangetttttttttt", "bangggeetttt", "bangttt"],
    "pokoknya": ["pokoknyaaaaa"],
    "yaudah": ["ydh"],
    "makasih": ["thxx"],
    "seler": ["sellerr"],
    "hihi": ["xixi", "xixixi", "wkwk"],
    "butuh": ["butuhhhh"],
    "kependekan": ["kependekannnnnn"],
    "gila": ["gilaaaakkkkkk"],
    "halo" : ["woeeeeee", "woiiiiiiiiiiii"],
    "parah": ["parahhhh"],
    "cuman": ["cumannn"],
    "sih": ["sihhhhh"],
    "suka": ["sukakkkkkk"],
    "oke": ["okeeeeeeeeeeeeeeeeeeeeeeeeeee"],
    "adem": ["ademmmmnnnn"],
    "seller": ["sellerrrr"],
    "lucu": ["gemoyyyyyy"],
    "cinta": ["luvvv"],
    "cantik": ["wapikk"],
    "harga": ["hargae"],
    "mahal": ["mehongg"],
    "sahabat": ["bestii"],
    "bunga": ["bukett"],
    "murah": ["murceeee"],
    "lama": ["lamaaaaaaaaaa"],
    "ya": ["yaaaa"],
    "tidak": ["gk", "ga", "gak"],
    "cuma": ["cuman"],
    "aduh": ["deg"],
    "apa": ["papa"],
    "taruh": ["taro"],
    "nya": ["y"],
    "pas": ["ps"],
    "sangat": ["sangatttt"],
    "dengan": ["dg"],
    "tahu": ["tau"],
    "itu": ["tuh"],
    "kakak": ["kak"],
    "bagaimana": ["gimana"],
    "benar": ["bner"],
    "lah": ["loh", "lho"],
    "banget": ["bgttt", "bgtttt"],
    "padahal": ["pdhl"],
    "aku": ["ku"],
    "juga": ["jg"],
    "sayang": ["syg"],
    "lebar": ["wide"],
    "kaki": ["leg"],
    "yang": ["yg"],
    "merek": ["merk"],
    "sudah": ["udah"]
}

def spilling(tokens):
    for i, line_token in enumerate(tokens):
        stop = 0
        for key in spill_words:
            for sp in spill_words[key]:
                if(line_token == sp):
                    tokens[i] = key
                    stop = 1
                    break
            if(stop == 1) : break
    return tokens

Normalization

In [ ]:
normalization_word = pd.read_csv('colloquial-indonesian-lexicon.csv')
normalization_word_dict = {}

for index, row in normalization_word.iterrows():
    if row[0] not in normalization_word_dict:
        normalization_word_dict[row[0]] = row[1]

def normalization(tokens):
    return [normalization_word_dict[term] if term in normalization_word_dict else term for term in tokens]

data['Normalization'] = data['Tokenize'].apply(normalization)

data['Normalization'] = data['Normalization'].apply(spilling)

from nltk.tokenize.treebank import TreebankWordDetokenizer
data['Normalization'] = data['Normalization'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))
data

for i in data['Normalization']:
    print(i);

In [82]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming(txt):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem = stemmer.stem(txt)
    print(stem)
    return stem

data['Stemming'] = data['Normalization'].apply(lambda txt: stemming(txt))
data

kemeja nya bagus banget mau nang kenapa enggak dari dulu beli kemeja toko ini ini kemeja asli memang bagus bahan juga adem enggak gerah and enggak terawang juga itu karena camera nya jelek jadi enggak terlalu jelas kemeja asliny bagus banget enggak bohong sumpah
jahit sih rapi cuma ada benang yang ikut ke jahit juga jadi agak jelek
sesuai harga agak tipis tapi masih oke kok warna abu tapi kalo foto memang kayak biru dikit thanks seler
wah gila sih bagus itu se worth itu se lembut itu baju ira bakal terlalu tipis kaya kemeja aku yang lama nyata sedikit kaya kaos baju bagus banget deh dengan harga segitu sih worth itu thankyou ya next bakal order lagi
kain nya bagus halus tapi kok di buka kotor ya warna putih lagi
bagus deh dengan harga segitu kain juga enggak tipis banget dan enggak tebal banget kalo putih memang harus pakai dalem biar enggak terawang bagus kirim juga enggak lama
harga mahal bahan tipis banget kirim lama tapi warna realpict maaf foto enggak sesuai karena sudah di cuci
p

,Reviews,Label,Case_Folding,Cleansing,Tokenize,Normalization,Stemming
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,kemeja nya bagusss bgtttt aaaa mauuu nngisssss...,"[kemeja, nya, bagusss, bgtttt, aaaa, mauuu, nn...",kemeja nya bagus banget mau nangis kenapa eng...,kemeja nya bagus banget mau nang kenapa enggak...
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0,"jahitannya sih rapi,cuman ada benang yang ikut...",jahitannya sih rapi cuman ada benang yang ikut...,"[jahitannya, sih, rapi, cuman, ada, benang, ya...",jahitannya sih rapi cuma ada benang yang ikut ...,jahit sih rapi cuma ada benang yang ikut ke ja...
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0,sesuai harga. agak tipis tapi masih oke kok. w...,sesuai harga agak tipis tapi masih oke kok war...,"[sesuai, harga, agak, tipis, tapi, masih, oke,...",sesuai harga agak tipis tapi masih oke kok war...,sesuai harga agak tipis tapi masih oke kok war...
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1,"wah gila sihhh sebagus itu, se worth it, se l...",wah gila sihhh sebagus itu se worth it se lemb...,"[wah, gila, sihhh, sebagus, itu, se, worth, it...",wah gila sih sebagus itu se worth itu se lembu...,wah gila sih bagus itu se worth itu se lembut ...
4,Kain nya bagus halus \nTapi kok di bukak koto...,0,kain nya bagus halus \ntapi kok di bukak koto...,kain nya bagus halus tapi kok di bukak kotor y...,"[kain, nya, bagus, halus, tapi, kok, di, bukak...",kain nya bagus halus tapi kok di buka kotor ya...,kain nya bagus halus tapi kok di buka kotor ya...
...,...,...,...,...,...,...,...
826,Terima kasih barang sudah sampai sesuai ukuran...,1,terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukuran...,"[terima, kasih, barang, sudah, sampai, sesuai,...",terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukur d...
827,Mantapp realpicttt bangttt tapi pengemasan nya...,1,mantapp realpicttt bangttt tapi pengemasan nya...,mantapp realpicttt bangttt tapi pengemasan nya...,"[mantapp, realpicttt, bangttt, tapi, pengemasa...",mantap realpicttt bangttt tapi pengemasan nya ...,mantap realpicttt bangttt tapi emas nya cuma p...
828,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1,"suka bgt sama tasnya, ga kayak tas local. kere...",suka bgt sama tasnya ga kayak tas local keren ...,"[suka, bgt, sama, tasnya, ga, kayak, tas, loca...",suka banget sama tasnya enggak kayak tas local...,suka banget sama tas enggak kayak tas local ke...
829,kualitas produk sangat baik. produk original. ...,1,kualitas produk sangat baik. produk original. ...,kualitas produk sangat baik produk original ha...,"[kualitas, produk, sangat, baik, produk, origi...",kualitas produk sangat baik produk original ha...,kualitas produk sangat baik produk original ha...


Stopword

In [83]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

def stopwords_removal(txt):
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    filter = stopword.remove(txt)
    return filter

data['Stopword'] = data['Stemming'].apply(lambda txt: stopwords_removal(txt))
data

,Reviews,Label,Case_Folding,Cleansing,Tokenize,Normalization,Stemming,Stopword
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,kemeja nya bagusss bgtttt aaaa mauuu nngisssss...,"[kemeja, nya, bagusss, bgtttt, aaaa, mauuu, nn...",kemeja nya bagus banget mau nangis kenapa eng...,kemeja nya bagus banget mau nang kenapa enggak...,kemeja nya bagus banget mau nang enggak dulu b...
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0,"jahitannya sih rapi,cuman ada benang yang ikut...",jahitannya sih rapi cuman ada benang yang ikut...,"[jahitannya, sih, rapi, cuman, ada, benang, ya...",jahitannya sih rapi cuma ada benang yang ikut ...,jahit sih rapi cuma ada benang yang ikut ke ja...,jahit sih rapi cuma benang ikut jahit jadi jelek
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0,sesuai harga. agak tipis tapi masih oke kok. w...,sesuai harga agak tipis tapi masih oke kok war...,"[sesuai, harga, agak, tipis, tapi, masih, oke,...",sesuai harga agak tipis tapi masih oke kok war...,sesuai harga agak tipis tapi masih oke kok war...,sesuai harga tipis masih oke kok warna abu kal...
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1,"wah gila sihhh sebagus itu, se worth it, se l...",wah gila sihhh sebagus itu se worth it se lemb...,"[wah, gila, sihhh, sebagus, itu, se, worth, it...",wah gila sih sebagus itu se worth itu se lembu...,wah gila sih bagus itu se worth itu se lembut ...,wah gila sih bagus se worth se lembut baju ira...
4,Kain nya bagus halus \nTapi kok di bukak koto...,0,kain nya bagus halus \ntapi kok di bukak koto...,kain nya bagus halus tapi kok di bukak kotor y...,"[kain, nya, bagus, halus, tapi, kok, di, bukak...",kain nya bagus halus tapi kok di buka kotor ya...,kain nya bagus halus tapi kok di buka kotor ya...,kain nya bagus halus kok buka kotor warna putih
...,...,...,...,...,...,...,...,...
826,Terima kasih barang sudah sampai sesuai ukuran...,1,terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukuran...,"[terima, kasih, barang, sudah, sampai, sesuai,...",terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukur d...,terima kasih barang sampai sesuai ukur seesuai...
827,Mantapp realpicttt bangttt tapi pengemasan nya...,1,mantapp realpicttt bangttt tapi pengemasan nya...,mantapp realpicttt bangttt tapi pengemasan nya...,"[mantapp, realpicttt, bangttt, tapi, pengemasa...",mantap realpicttt bangttt tapi pengemasan nya ...,mantap realpicttt bangttt tapi emas nya cuma p...,mantap realpicttt bangttt emas nya cuma plasti...
828,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1,"suka bgt sama tasnya, ga kayak tas local. kere...",suka bgt sama tasnya ga kayak tas local keren ...,"[suka, bgt, sama, tasnya, ga, kayak, tas, loca...",suka banget sama tasnya enggak kayak tas local...,suka banget sama tas enggak kayak tas local ke...,suka banget sama tas enggak kayak tas local ke...
829,kualitas produk sangat baik. produk original. ...,1,kualitas produk sangat baik. produk original. ...,kualitas produk sangat baik produk original ha...,"[kualitas, produk, sangat, baik, produk, origi...",kualitas produk sangat baik produk original ha...,kualitas produk sangat baik produk original ha...,kualitas produk sangat baik produk original ha...


In [ ]:
# for i in data["Stopword"]:
#     print(i)

### Simpan Preprocessing

In [28]:
# nama_file = 'preprocessing_review.xlsx'  # Nama file Excel yang diinginkan
# data.to_excel(nama_file, index=False)  # Menyimpan DataFrame ke file Excel

# print("Data berhasil dikonversi ke file Excel.")

# texts = df["cerita"].tolist()
# labels = df["genre"].tolist()

# data = df[['cerita', 'genre']]
# ==================================
data = pd.read_excel("preprocessing_review.xlsx")
data

,Reviews,Label,Case_Folding,Cleansing,Tokenize,Normalization,Stemming,Stopword
0,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,kemeja nya bagusss bgtttt aaaa mauuu nngisssss...,"['kemeja', 'nya', 'bagusss', 'bgtttt', 'aaaa',...",kemeja nya bagus banget mau nangis kenapa eng...,kemeja nya bagus banget mau nang kenapa enggak...,kemeja nya bagus banget mau nang enggak dulu b...
1,"Jahitannya sih rapi,cuman ada benang yang ikut...",0,"jahitannya sih rapi,cuman ada benang yang ikut...",jahitannya sih rapi cuman ada benang yang ikut...,"['jahitannya', 'sih', 'rapi', 'cuman', 'ada', ...",jahitannya sih rapi cuma ada benang yang ikut ...,jahit sih rapi cuma ada benang yang ikut ke ja...,jahit sih rapi cuma benang ikut jahit jadi jelek
2,Sesuai harga. Agak tipis tapi masih oke kok. W...,0,sesuai harga. agak tipis tapi masih oke kok. w...,sesuai harga agak tipis tapi masih oke kok war...,"['sesuai', 'harga', 'agak', 'tipis', 'tapi', '...",sesuai harga agak tipis tapi masih oke kok war...,sesuai harga agak tipis tapi masih oke kok war...,sesuai harga tipis masih oke kok warna abu kal...
3,"Wah gila sihhh sebagus itu, se worth it, se l...",1,"wah gila sihhh sebagus itu, se worth it, se l...",wah gila sihhh sebagus itu se worth it se lemb...,"['wah', 'gila', 'sihhh', 'sebagus', 'itu', 'se...",wah gila sih sebagus itu se worth itu se lembu...,wah gila sih bagus itu se worth itu se lembut ...,wah gila sih bagus se worth se lembut baju ira...
4,Kain nya bagus halus \nTapi kok di bukak koto...,0,kain nya bagus halus \ntapi kok di bukak koto...,kain nya bagus halus tapi kok di bukak kotor y...,"['kain', 'nya', 'bagus', 'halus', 'tapi', 'kok...",kain nya bagus halus tapi kok di buka kotor ya...,kain nya bagus halus tapi kok di buka kotor ya...,kain nya bagus halus kok buka kotor warna putih
...,...,...,...,...,...,...,...,...
826,Terima kasih barang sudah sampai sesuai ukuran...,1,terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukuran...,"['terima', 'kasih', 'barang', 'sudah', 'sampai...",terima kasih barang sudah sampai sesuai ukuran...,terima kasih barang sudah sampai sesuai ukur d...,terima kasih barang sampai sesuai ukur seesuai...
827,Mantapp realpicttt bangttt tapi pengemasan nya...,1,mantapp realpicttt bangttt tapi pengemasan nya...,mantapp realpicttt bangttt tapi pengemasan nya...,"['mantapp', 'realpicttt', 'bangttt', 'tapi', '...",mantap realpicttt bangttt tapi pengemasan nya ...,mantap realpicttt bangttt tapi emas nya cuma p...,mantap realpicttt bangttt emas nya cuma plasti...
828,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1,"suka bgt sama tasnya, ga kayak tas local. kere...",suka bgt sama tasnya ga kayak tas local keren ...,"['suka', 'bgt', 'sama', 'tasnya', 'ga', 'kayak...",suka banget sama tasnya enggak kayak tas local...,suka banget sama tas enggak kayak tas local ke...,suka banget sama tas enggak kayak tas local ke...
829,kualitas produk sangat baik. produk original. ...,1,kualitas produk sangat baik. produk original. ...,kualitas produk sangat baik produk original ha...,"['kualitas', 'produk', 'sangat', 'baik', 'prod...",kualitas produk sangat baik produk original ha...,kualitas produk sangat baik produk original ha...,kualitas produk sangat baik produk original ha...


TF-IDF

In [29]:
# pip install scikit-learn

import sklearn
sklearn.__version__

'1.2.2'

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(data['Stopword']).toarray()
y = data.iloc[:, 1].values
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [31]:
from sklearn.feature_selection import SelectKBest, chi2

# Menggunakan 100 fitur terbaik berdasarkan metode Chi-square
# selector = SelectKBest(chi2, k=int(0.1 * X.shape[1])) # 93%
# selector = SelectKBest(chi2, k=int(0.2 * X.shape[1])) # 95%
# selector = SelectKBest(chi2, k=int(0.3 * X.shape[1])) # 93%
selector = SelectKBest(chi2, k=int(0.4 * X.shape[1])) # 94%
X = selector.fit_transform(X, y)

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
print(X)

# Tampilkan matriks TF-IDF dengan term-nya
# for row in X:
#     print(" ".join("{:.4f}".format(value) for value in row))


[[0.         0.         0.14545139 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.31065643 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


GridSearch

In [33]:
# from sklearn.model_selection import GridSearchCV
# from sklearn import svm
# svm = svm.SVC()

# param = {
#     'C': (0.1, 1, 10, 100),
#     'kernel': ('linear', 'rbf', 'sigmoid', 'poly'),
#     'gamma': (0.0001, 0.001, 0.1, 1, 'scale', 'auto'),
#     'degree': (2, 3, 4)
#     }

# # nilai gamma = 0.0001, 0.001, 0.1, atau 1

# grid = GridSearchCV(svm, param)
# grid.fit(X,y)
# grid.best_params_

In [34]:
# grid.best_score_

Cross Validation

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

# clf = svm.SVC(C=1, degree=2, gamma='scale', kernel='sigmoid')
# clf = svm.SVC(C=10, degree=2, gamma='scale', kernel='linear')
clf = svm.SVC(C=10, degree=2, gamma=0.1, kernel='rbf')
scores = cross_val_score(clf, X, y, cv=5)

print(scores)
scores.mean()

[0.91017964 0.91566265 0.92771084 0.89759036 0.95180723]


0.9205901450111824

Split Dataset

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=y, random_state=42)

Klasifikasi SVM

In [37]:
from sklearn import svm

classifierSVM = svm.SVC(C=10, degree=2, gamma=0.1, kernel='rbf')
# classifierSVM = svm.SVC(C=10, degree=2, gamma='scale', kernel='linear')
# classifierSVM = svm.SVC(C=1, degree=2, gamma=0.001, kernel='linear')
# training
classifierSVM.fit(X_train, y_train)
# prediksi data test

y_pred_SVM = classifierSVM.predict(X_test)
print(y_pred_SVM)

[0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0
 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1
 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 0 1
 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0]


Uji Performansi

In [44]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_SVM))
print(confusion_matrix(y_test, y_pred_SVM))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        84
           1       1.00      0.88      0.94        83

    accuracy                           0.94       167
   macro avg       0.95      0.94      0.94       167
weighted avg       0.95      0.94      0.94       167

[[84  0]
 [10 73]]


In [39]:
from sklearn.metrics import precision_score, recall_score, f1_score

labels = [0, 1]

pre = precision_score(y_test, y_pred_SVM, average='macro', labels=labels)
print("Precision : ", pre)

rec = recall_score(y_test, y_pred_SVM, average='macro', labels=labels)
print("Recall : ", rec)

f1 = f1_score(y_test, y_pred_SVM, average='macro', labels=labels)
print("F1-Score : ", f1)
print()
pre = precision_score(y_test, y_pred_SVM, average='weighted', labels=labels)
print("Precision : ", pre)

rec = recall_score(y_test, y_pred_SVM, average='weighted', labels=labels)
print("Recall : ", rec)

f1 = f1_score(y_test, y_pred_SVM, average='weighted', labels=labels)
print("F1-Score : ", f1)


Precision :  0.9468085106382979
Recall :  0.9397590361445782
F1-Score :  0.9398588303082684

Precision :  0.9464899987259524
Recall :  0.9401197604790419
F1-Score :  0.9398825512328842


#### Save & Load Model

In [40]:
import pickle

# filename = "model/SVM_10_0-1_rbf.pkl"

# with open(filename, 'wb') as file:
#     pickle.dump(classifierSVM, file)

In [41]:
# filepath = "model/SVM_10_0-1_rbf.pkl"
# with open(filepath, 'rb') as file:
#     classifierSVM2 = pickle.load(file)

### Prediksi

In [42]:
text = np.array([
    'Produk yang saya beli dari toko online ini sangat keren', 
    'Saya sangat puas dengan pengiriman produk ini Waktu pengirimannya cepat dan packingnya sangat rapi.', 
    'Produk ini benar-benar memenuhi harapan saya. Kualitasnya sangat baik, harga terjangkau, dan layanan pelanggan yang responsif.', 
    'Saya sangat kecewa dengan produk yang saya beli. Ketika tiba, barangnya dalam kondisi rusak dan tidak sesuai dengan deskripsi yang diberikan.', 
    'Pengiriman produk ini sangat lambat. Saya harus menunggu berhari-hari untuk mendapatkan barangnya, padahal saya telah membayar biaya pengiriman yang mahal.', 
    'Saya merasa tertipu oleh gambar produk yang ditampilkan. Ketika produk tiba, warna dan kualitasnya jauh berbeda dari yang ditampilkan di situs jual beli online.'
    ], dtype=object)

sample = cv.transform(text)
sample_selected = selector.transform(sample).toarray()
sample_selected = tfidfconverter.transform(sample_selected).toarray()

predicted = classifierSVM.predict(sample_selected)
print(predicted)

[1 1 1 0 0 0]
